# Try to detect null samples using reverse PCA

In [1]:

%load_ext autoreload
%autoreload 2
from sklearn.preprocessing import StandardScaler
from preprocessing._chunk_preparation import concat_chunks_for_feature_extraction, preprocess_chunks_for_null_test_with_indoor
from preprocessing._segmentation import segment_for_null_classification

from data_reading.phyphox import read_experiments_in_dir
from features import extract_timeseries_features
from file_handling import get_sub_directories
from tsfresh.feature_extraction.settings import MinimalFCParameters, ComprehensiveFCParameters, EfficientFCParameters

sample_rate = 50

# config:

experiment_dir_path = "../../data/phyphox/full recordings/"
experiment_dirs = get_sub_directories(experiment_dir_path)
use_indoor = True
window_size = 50
# MinimalFCParameters, ComprehensiveFCParameters, EfficientFCParameters
feature_calculation_setting = MinimalFCParameters()

# Ana-2, Ariane, Julian, Wiki
experiment_dirs_selected = ["Ana-2","Anne","Ariane","Cilly","Fabi","Julian","Julius","Wiktoria"]
#experiment_dirs_selected = ["Julius"]
experiment_dirs = [exp_dir for exp_dir in experiment_dirs if exp_dir.split("/")[-1] in experiment_dirs_selected]

chunks, null_chunks, y = read_experiments_in_dir(experiment_dirs, sample_rate, drop_lin_acc=True)
chunks_ocd, chunks_null_class = preprocess_chunks_for_null_test_with_indoor(chunks, null_chunks)
chunks_ocd_segmented, labels_ocd_segmented, chunks_null_segmented, labels_null_segmented = segment_for_null_classification(chunks_ocd, chunks_null_class, window_size)

null_class_df, _ = concat_chunks_for_feature_extraction(
        [chunks_null_segmented],
        [labels_null_segmented])

# without null class
ocd_df, _ = concat_chunks_for_feature_extraction(
        [chunks_ocd_segmented],
        [labels_ocd_segmented])
assert len(set(labels_ocd_segmented)) == 1
assert len(set(labels_null_segmented)) == 1

/Users/julio/Documents/Uni/5_Master/Master Project/mp-mental-health-2020/src/preprocessing/_data_frame.py:31: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  time_delta_index = pd.TimedeltaIndex(timestamp_to_date, unit=output_timestamp_unit)
/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/julio/Documents/Uni/5_Master/Master Project/mp-mental-health-2020/src/preprocessing/_segmentation.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

In [ ]:
X_ocd = extract_timeseries_features(ocd_df, use_indoor=use_indoor,use_fingerprinting_approach=True, feature_set_config=feature_calculation_setting)
X_null_class = extract_timeseries_features(null_class_df, use_indoor=use_indoor,use_fingerprinting_approach=True, feature_set_config=feature_calculation_setting)

scaler = StandardScaler()
X_multi_class_classification_scaled = scaler.fit_transform(X_ocd)
X_null_class_classification_scaled = scaler.fit_transform(X_null_class)
%matplotlib qt

from visualization._visualization import pca_2d_null_clf
pca_2d_null_clf(X_multi_class_classification_scaled, X_null_class_classification_scaled)